<center>
    <img src="./share&code.png" width="50%" height="50%">
</center>

# Traits in Julia and WhereTraits.jl

my name is Stephan Sahm, s.sahm@reply.de, author of WhereTraits.jl

# Why Traits?

**TLDR:** Julia's type system has no multiple inheritance. There is only one taxonomy.

But sometimes we want to dispatch on other dimensions.

<center>
    <img src="https://images.unsplash.com/photo-1580604056714-13f7e75d4c84?ixlib=rb-1.2.1&ixid=MXwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHw%3D&auto=format&fit=crop&w=2250&q=80" width="80%" height="50%">
</center>

Example Taxonomy

In [1]:
abstract type Animal end
abstract type Mammal <: Animal end

struct Whale <: Mammal
    name
end

struct CyanoBacteria <: Animal
    name
end

In [2]:
name(animal::Animal) = animal.name

moby = Whale("Moby-Dick")
pantoffel = CyanoBacteria("Pantoffel")

name(moby), name(pantoffel)

("Moby-Dick", "Pantoffel")

# Pure Julia Traits

You just need 2 ingredients

1) define a parallel taxonomy

In [3]:
abstract type Locomotion end
struct Swimming <: Locomotion end
struct Walking <: Locomotion end

const swimming = Swimming()
const walking = Walking()

Walking()

2) defining a mapping from the one taxonomy to the other

In [4]:
Locomotion(::Whale) = swimming
Locomotion(::CyanoBacteria) = swimming

Locomotion

Now you can dispatch on the orthogonal taxonomy (aka traits)

In [21]:
move(animal::Animal) = move(Locomotion(animal), animal)

move (generic function with 3 methods)

In [22]:
move(::Swimming, animal) = "Just keep swimming $(name(animal))!"
move(::Walking, animal) = "Run $(name(animal)) run!"

move (generic function with 5 methods)

In [23]:
move(pantoffel)

"Just keep swimming Pantoffel!"

Alternatively you could split the taxonomy into several, by just having binary traits.

In [24]:
canswim(::Whale) = true
canswim(::CyanoBacteria) = true

canwalk(::Whale) = false
canwalk(::CyanoBacteria) = false

canwalk (generic function with 2 methods)

The dispatch looks slightly more complex, because Julia `true`/`false` need to be wrapped into `Val` to be ready for dispatch.

In [29]:
move2(animal::Animal) = move2(animal, Val(canswim(animal)), Val(canwalk(animal)))
move2(animal, _swim::Val{true}, _walk::Val{true}) = "First walk, then swim."
move2(animal, _swim::Val{true}, _walk::Val{false}) = "Swim"
move2(animal, _swim::Val{false}, _walk::Val{true}) = "Walk"
move2(animal, _swim::Val{false}, _walk::Val{false}) = "just stand around"

move2 (generic function with 5 methods)

In [31]:
move2(moby)

"Swim"

Natural Error support

In [32]:
struct Cat <: Mammal
    name
end
kitty = Cat("Kitty")

Cat("Kitty")

In [33]:
move(kitty)

LoadError: MethodError: no method matching Locomotion(::Cat)
Closest candidates are:
  Locomotion(!Matched::Whale) at In[4]:1
  Locomotion(!Matched::CyanoBacteria) at In[4]:2
  Locomotion(!Matched::Type{Mammal}) at In[18]:1

In [13]:
move2(kitty)

LoadError: MethodError: no method matching canswim(::Cat)
Closest candidates are:
  canswim(!Matched::CyanoBacteria) at In[8]:2
  canswim(!Matched::Whale) at In[8]:1

Most typical application of traits:
1. built functions for general interfaces (e.g. iterables)
2. specialize these functions further for subcases

# WhereTraits.jl

- intuitive syntax for pure julia traits, merging both levels
- easily extendable

In [14]:
using WhereTraits

In [46]:
@traits function move3(animal::Animal) where {canswim(animal)}
    error("my inner error")
    return "take the flow"    
end
@traits move3(animal::Animal, args...) where {canwalk(args...)} = "step by step"
@traits move3(animal::Animal) where {canswim(animal), canwalk(animal)} = "first swim then walk"
@traits move3(animal::Animal) = "fallback"

In [38]:
move3(pantoffel)

"take the flow"

In [48]:
canswim(::Whale) = true
canwalk(::Whale) = false
move3(moby)

LoadError: my inner error